In [2]:
import fastf1
import pandas as pd
import os

fastf1.Cache.enable_cache('../cache')

# Drivers to compare
drivers = {'NOR': 'Lando Norris', 'PIA': 'Oscar Piastri'}
# Seasons to compare
seasons = [2024, 2025]

# Storage
records = []

# Ensure cache and directories exist
GRAPH_DIR = "graphs"
os.makedirs(GRAPH_DIR, exist_ok=True)

# --------------------------
# Collect quali & race results
# --------------------------
for season in seasons:
    schedule = fastf1.get_event_schedule(season, force_ergast=True)
    
    for _, race_event in schedule.iterrows():
        round_ = race_event["RoundNumber"]
        race_name = race_event["EventName"]

        try:
            # Load qualifying and race sessions
            quali = fastf1.get_session(season, round_, 'Q')
            quali.load(telemetry=False)

            race = fastf1.get_session(season, round_, 'R')
            race.load(telemetry=False)
        except:
            continue

        for drv_abbr in drivers:
            quali_pos = None
            finish_pos = None

            # Get qualifying position
            qrow = quali.results[quali.results["Abbreviation"] == drv_abbr]
            if not qrow.empty:
                quali_pos = int(qrow["Position"].values[0])

            # Get race finish position
            frow = race.results[race.results["Abbreviation"] == drv_abbr]
            if not frow.empty:
                finish_pos = int(frow["Position"].values[0])

            # Podium & win
            podium = finish_pos in [1, 2, 3] if finish_pos is not None else False
            win = finish_pos == 1 if finish_pos is not None else False

            # Average race lap time
            avg_lap_time = None
            laps = race.laps.pick_driver(drv_abbr).LapTime
            if len(laps) > 0:
                avg_lap_time = laps.dt.total_seconds().mean()

            # Append to records
            records.append({
                "Season": season,
                "Race": race_name,
                "Driver": drivers[drv_abbr],
                "QualiPos": quali_pos,
                "FinishPos": finish_pos,
                "Podium": podium,
                "Win": win,
                "AvgLapTime_s": avg_lap_time
            })

# Save raw race-level data
df = pd.DataFrame(records)
df.to_csv("lando_oscar_race_data_2024_2025.csv", index=False)


c:\Users\raima\Desktop\F1 analysis\venv\Lib\site-packages\fastf1\events.py:310: UserWarning: Option ``force_ergast`` has been deprecated, use``backend='ergast'`` instead
  warnings.warn("Option ``force_ergast`` has been deprecated, use"
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24'